## Simple ETL / Exploration with node-rapids

This notebook will demonstrate how basic APIs from `node-rapids` ([GitHub](https://github.com/rapidsai/node-rapids), [docs](https://rapidsai.github.io/node-rapids/)) may be used to load and process data from the GPU in Node.

First, we load the cudf module from `node-rapids`:

In [1]:
cudf = require("@rapidsai/cudf");

{
  Column: [Function: Column],
  DataFrame: [class DataFrame],
  GroupByMultiple: [class GroupByMultiple extends GroupByBase],
  GroupBySingle: [class GroupBySingle extends GroupByBase],
  AbstractSeries: [class AbstractSeries],
  Series: [class AbstractSeries],
  Bool8Series: [class Bool8Series extends NumericSeries],
  Float32Series: [class Float32Series extends FloatSeries],
  Float64Series: [class Float64Series extends FloatSeries],
  Int8Series: [class Int8Series extends IntSeries],
  Int16Series: [class Int16Series extends IntSeries],
  Int32Series: [class Int32Series extends IntSeries],
  Uint8Series: [class Uint8Series extends IntSeries],
  Uint16Series: [class Uint16Series extends IntSeries],
  Uint32Series: [class Uint32Series extends IntSeries],
  Int64Series: [class Int64Series extends IntSeries],
  Uint64Series: [class Uint64Series extends IntSeries],
  StringSeries: [class StringSeries extends AbstractSeries],
  ListSeries: [class ListSeries extends AbstractSeries],
  St

We are going to look at the 1.5 Gb [US Accidents (Dec 20) dataset from Kaggle](https://www.kaggle.com/sobhanmoosavi/us-accidents?select=US_Accidents_Dec20.csv). First we need to define the columns and associated datatypes for this table:

In [2]:
columns =  {
    id: 'str', source: 'str', tmc: 'float64', severity: 'int32', start_time: 'str', end_time: 'str',
    start_lat: 'float64', start_lng: 'float64', end_lat: 'float64', end_lng: 'float64',
    distance: 'float64', description: 'str', number: 'int32', street: 'str', side: 'str',
    city: 'str', county: 'str', state: 'str', zipcode: 'str', country: 'str', timezone: 'str', airport_code: 'str', 
    weather_timestamp: 'str', temperature: 'float64', wind_chill: 'float64', humidity: 'float64', pressure: 'float64', 
    visibility: 'float64', wind_direction: 'str', wind_speed: 'float64', precipitation: 'float64', weather_condition: 'str', 
    amenity: 'bool', bump: 'bool', crossing: 'bool', give_way: 'bool', junction: 'bool', no_exit: 'bool', railway: 'bool', 
    roundabout: 'bool', station: 'bool', stop: 'bool', traffic_calming: 'bool', traffic_signal: 'bool', turning_loop: 'bool', 
    sunrise_sunset: 'str', civil_twilight: 'str', nautical_twighlight: 'str', astronomical_twighlight: 'str'
}

{
  id: 'str',
  source: 'str',
  tmc: 'float64',
  severity: 'int32',
  start_time: 'str',
  end_time: 'str',
  start_lat: 'float64',
  start_lng: 'float64',
  end_lat: 'float64',
  end_lng: 'float64',
  distance: 'float64',
  description: 'str',
  number: 'int32',
  street: 'str',
  side: 'str',
  city: 'str',
  county: 'str',
  state: 'str',
  zipcode: 'str',
  country: 'str',
  timezone: 'str',
  airport_code: 'str',
  weather_timestamp: 'str',
  temperature: 'float64',
  wind_chill: 'float64',
  humidity: 'float64',
  pressure: 'float64',
  visibility: 'float64',
  wind_direction: 'str',
  wind_speed: 'float64',
  precipitation: 'float64',
  weather_condition: 'str',
  amenity: 'bool',
  bump: 'bool',
  crossing: 'bool',
  give_way: 'bool',
  junction: 'bool',
  no_exit: 'bool',
  railway: 'bool',
  roundabout: 'bool',
  station: 'bool',
  stop: 'bool',
  traffic_calming: 'bool',
  traffic_signal: 'bool',
  turning_loop: 'bool',
  sunrise_sunset: 'str',
  civil_twilight: 'str',
  

In [3]:
console.time("readCSV")
df = cudf.DataFrame.readCSV({
    header: 0,
    sourceType: 'files',
    sources: ["data/US_Accidents_Dec20.csv"],
    dataTypes: columns,
});
console.timeEnd("readCSV")

readCSV: 2.071s


Now that we have loaded the CSV into a GPU DataFrame `df` we can look at some basic information like number of rows and columns:

In [4]:
console.log("Number of rows:", df.numRows)
console.log("Number of cols:", df.numColumns)

Number of rows: 4232541
Number of cols: 49


This data set has lots of columns we don't really care about. We can pare things down using the `Datafame.drop` method:

In [5]:
df = df.drop([
    'civil_twilight', 'nautical_twighlight', 'astronomical_twighlight', 'wind_speed', 'wind_direction', 'wind_chill', 'humidity', 
    'pressure', 'amenity', 'bump', 'give_way', 'no_exit', 'railway', 'roundabout', 'station', 'traffic_calming', 'turning_loop', 'timezone', 
    'airport_code', 'tmc', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'street', 'country', 'distance']
)

DataFrame {
  _accessor: ColumnAccessor {
    _labels_to_indices: Map(20) {
      'id' => 0,
      'source' => 1,
      'severity' => 2,
      'description' => 3,
      'number' => 4,
      'side' => 5,
      'city' => 6,
      'county' => 7,
      'state' => 8,
      'zipcode' => 9,
      'weather_timestamp' => 10,
      'temperature' => 11,
      'visibility' => 12,
      'precipitation' => 13,
      'weather_condition' => 14,
      'crossing' => 15,
      'junction' => 16,
      'stop' => 17,
      'traffic_signal' => 18,
      'sunrise_sunset' => 19
    },
    _data: {
      id: Column {},
      source: Column {},
      severity: Column {},
      description: Column {},
      number: Column {},
      side: Column {},
      city: Column {},
      county: Column {},
      state: Column {},
      zipcode: Column {},
      weather_timestamp: Column {},
      temperature: Column {},
      visibility: Column {},
      precipitation: Column {},
      weather_condition: Column {},
      cr

In [6]:
df.names

[
  'id',                'source',
  'severity',          'description',
  'number',            'side',
  'city',              'county',
  'state',             'zipcode',
  'weather_timestamp', 'temperature',
  'visibility',        'precipitation',
  'weather_condition', 'crossing',
  'junction',          'stop',
  'traffic_signal',    'sunrise_sunset'
]

In [7]:
temp = df.get('temperature')
console.log("Min temp:", temp.min())
console.log("Max temp:", temp.max())

Min temp: -89
Max temp: 203


Some of the temperature values are clearly bad data, let's restrict the datafame to a more reasonable range. The `lt` and `gt` unary operators return a boolean mask where values are less or greater than ven values, respectively. These masks can be combined with the `logical_or` operator and then passed to `DataFrame.gather` to restrict to only the valid rows we care about:

In [8]:
temp = df.get('temperature')

console.time("filter")
valid_temps = temp.lt(120).logical_and(temp.gt(-30))
df = df.filter(valid_temps)
console.timeEnd("filter")

filter: 156.711ms


We can see above how long filtering the full 1.5 Gb data set took. Below we can verify that that filtered data only has values in the specified range:

In [9]:
temp = df.get('temperature')

console.log("New number of rows:", df.numRows)
console.log("New min temp:", temp.min())
console.log("New max temp:", temp.max())

New number of rows: 4142582
New min temp: -29.9
New max temp: 119


Another thing we might want to examine is the grouping of weather conditions. The original dataframe has very fine-grained weather conditions. e.g "Fog" vd "Shallow Fog", as seen below:

In [10]:
weather_groups = df.groupBy({by: "weather_condition"})
JSON.stringify(weather_groups.nth(0).get("weather_condition").toArrow().toArray())

'["Blowing Dust","Blowing Dust / Windy","Blowing Sand","Blowing Snow","Blowing Snow / Windy","Clear","Cloudy","Cloudy / Windy","Drifting Snow","Drizzle","Drizzle / Windy","Drizzle and Fog","Dust Whirls","Fair","Fair / Windy","Fog","Fog / Windy","Freezing Drizzle","Freezing Rain","Freezing Rain / Windy","Funnel Cloud","Hail","Haze","Haze / Windy","Heavy Blowing Snow","Heavy Drizzle","Heavy Freezing Drizzle","Heavy Freezing Rain","Heavy Ice Pellets","Heavy Rain","Heavy Rain / Windy","Heavy Rain Shower","Heavy Rain Showers","Heavy Sleet","Heavy Smoke","Heavy Snow","Heavy Snow / Windy","Heavy Snow with Thunder","Heavy T-Storm","Heavy T-Storm / Windy","Heavy Thunderstorms and Rain","Heavy Thunderstorms and Snow","Heavy Thunderstorms with Small Hail","Ice Pellets","Light Blowing Snow","Light Drizzle","Light Drizzle / Windy","Light Fog","Light Freezing Drizzle","Light Freezing Fog","Light Freezing Rain","Light Freezing Rain / Windy","Light Hail","Light Haze","Light Ice Pellets","Light Rain","

Let's use Cudf's GPU regex functions to get some quick counts of more generic weather categories. The `Series.containsRe` method will return a boolean mask that is true wherever the series value matches the regex:

In [11]:
weather = df.get("weather_condition")

console.time("regex")
clouds_mask = weather.containsRe("Cloud|Overcast");
rain_mask = weather.containsRe("Rain|T-Storm|Thunderstorm|Squalls|Drizzle");
snow_mask = weather.containsRe("Snow")
fog_mask = weather.containsRe("Fog")
ice_mask = weather.containsRe("Ice|Hail|Freezing|Sleet")
particulate_mask = weather.containsRe("Dust|Smoke|Sand")
console.timeEnd("regex")

regex: 185.819ms


The categorization above is not necessarily exlcusive, and categories may overlap, but we can see how many accidents had a category involved by summing each mask:

In [14]:
console.time("sum")
console.log("Severity with clouds     :", clouds_mask.sum())
console.log("Severity with rain       :", rain_mask.sum())
console.log("Severity with snow       :", snow_mask.sum())
console.log("Severity with fog        :", fog_mask.sum())
console.log("Severity with particulate:", particulate_mask.sum())
console.log("Severity with ice        :", ice_mask.sum())
console.timeEnd("sum")

Severity with clouds     : 1890261
Severity with rain       : 325213
Severity with snow       : 67961
Severity with fog        : 51781
Severity with particulate: 8795
Severity with ice        : 4689
sum: 23.25ms


We might be interested to look filter by these subsets to see the average severity when each category is involved:

In [15]:
console.time("means")
console.log("Severity with clouds     :", df.filter(clouds_mask).get("severity").mean())
console.log("Severity with rain       :", df.filter(rain_mask).get("severity").mean())
console.log("Severity with snow       :", df.filter(snow_mask).get("severity").mean())
console.log("Severity with fog        :", df.filter(fog_mask).get("severity").mean())
console.log("Severity with particulate:", df.filter(particulate_mask).get("severity").mean())
console.log("Severity with ice        :", df.filter(ice_mask).get("severity").mean())
console.timeEnd("means")


Severity with clouds     : 2.3207297828183515
Severity with rain       : 2.351904751655069
Severity with snow       : 2.4022012624887803
Severity with fog        : 2.2152720109692745
Severity with particulate: 2.2823194997157477
Severity with ice        : 2.4768607378972063
means: 156.491ms


Unsurprisingly, the most severe accidents were recorded in ice and snow conditions.

Hopefully this has been a helpful introduction to Cudf in node-rapids! For more information [see the documentation](https://rapidsai.github.io/node-rapids/).